In [ ]:
from aiida import load_profile,orm
load_profile();
import ipywidgets as widgets
from IPython.display import display
from widgets import sens_query, inversion_params
import aiidalab_widgets_base as awb
from settings import *

# Inversion

### Import data

In [ ]:
style = {"description_width": "initial"}
address = widgets.Text(
            placeholder = 'Address to files',
            description="Remote address",
            style=style,
        )
def prepare_inspect():
    computer = orm.load_computer(label='daint')
    remote_path = orm.RemoteData(remote_path = address.value,
                             computer = computer)
    builder = INSPECT.get_builder()
    builder.remotes = {'a':remote_path,}
                      #'b':remote_path_2}
    return builder

btn_submit_1 = awb.SubmitButtonWidget(
    INSPECT,
    inputs_generator = prepare_inspect,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_1.btn_submit.button_style="info"
btn_submit_1.btn_submit.description="Launch import"

line_h = widgets.HTML(
            value="""<hr>
            """
        )
info_text = widgets.HTML(
            value="""Find for nc lise in stash folder of previous 
            Collect sensitivities calclations.
            """
        )

def prepare_inspect_cs():
    qb = orm.QueryBuilder()
    qb.append(COLLECT_SENS, tag="cs", 
              filters={"attributes.exit_status": 0}
              )
    qb.append(orm.RemoteStashFolderData, with_incoming="cs", project="*")
    builder = INSPECT.get_builder()
    directories_ = {f'test_{j}':i[0] for j,i in enumerate(qb.all())}
    if directories_:
        builder.remotes_cs = directories_
        return builder
    else:
        print('No files found.')

btn_submit_2 = awb.SubmitButtonWidget(
    INSPECT,
    inputs_generator = prepare_inspect_cs,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_2.btn_submit.button_style="success"
btn_submit_2.btn_submit.description="Launch CS import"

acc = widgets.Accordion(
            children=[
                 widgets.VBox(
                    children=[
                address,
                btn_submit_1,
                line_h,
                info_text,
                btn_submit_2
                    ])
            ],
        )
acc.set_title(0, "Import")
acc.selected_index = None
acc

###  Search

In [ ]:
sq = sens_query.SearchSens()
sq

In [ ]:
inversion_button = widgets.Button(description = 'Submit Inversion',
                                  button_style = "success")
reset_button = widgets.Button(description = 'Reset',
                                  button_style = "info")
out = widgets.Output()

def on_click_reset(b):
    sq.list_remotes = []
    sq.available_obs_list['names'] = []
    sq.available_obs_list['remotes'] = []

def on_click(b):
    out.clear_output()
    with out:
        print('AVAILABLE\n')
        for i in set(sq.list_remotes):
            print(f'\t - {i}')
        print('\nOBSERVATIONS\n')
        for i in set(sq.available_obs_list['remotes']):
            print(f'\t - {i}')

inversion_button.on_click(on_click)
reset_button.on_click(on_click_reset)

### Step 2

In [ ]:
inv = inversion_params.InversionParams()
inv


In [ ]:
"""import yaml
params_dict = inv.construct_dict()
with open('inv_params.yaml', 'w') as f:
    _ = yaml.dump(params_dict, f)"""

### Step 3

In [ ]:
display(inversion_button,reset_button,out)